In [ ]:
import pandas as pd
import requests
from playwright.async_api import async_playwright
import zlib
from pathway_indexer.parser import parse_files_to_md

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()


In [ ]:
def generate_hash_filename(url):
    """Generate a hash of the URL to use as a filename."""
    url_hash = zlib.crc32(url.encode())
    file_name = f"{url_hash:x}"
    return file_name


In [ ]:
save_path = "../../data/data_01_11"
url = "https://catalog.byupathway.edu/programs?type=Degree&page=1&pq="

In [ ]:
await page.goto(url)
await page.wait_for_load_state()


In [ ]:
while True:
    try:
        await page.click("text=Show More...")
        button = page.query_selector("#pagingButton")
    except:
        break


In [ ]:
# get the element with id articleList
articles = await page.query_selector("#programs-list")
# get all the links inside it with data-test="programCard"
# links = await articles.query_selector_all('a[data-test="programCard"]')
# data = []
# for link in links:
#     print(link)
#     title = await link.query_selector("h3.media-title")
#     filename = generate_hash_filename(url + await link.get_attribute("href"))
#     data.append(
#         {
#             "URL": url + await link.get_attribute("href"),
#             "Title": await title.inner_text(),
#             "filename": filename,
#         }
#     )

# df = pd.DataFrame(data)
# df

In [33]:
print(articles)

None


In [ ]:
# save the df in a csv file
df.to_csv(f"{save_path}/articles.csv", index=False)

In [ ]:
for index, row in df.iterrows():
    await page.goto(row["URL"])
    await page.wait_for_load_state('networkidle')  # Espera hasta que la red esté inactiva
    content = await page.query_selector('[role="main"]#main-content')
    if content:
        post_content = await content.inner_html()
        if post_content:
            with open(f"{save_path}/crawl/{row['filename']}.html", "w") as f:
                f.write(post_content)